In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import pandas as pd

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [52]:
X = pd.read_csv('../data/processed/X_train', header=0, index_col=0)
X_final_test = pd.read_csv('../data/processed/X_test', header=0, index_col=0)
y = pd.read_csv('../data/processed/y_train', header=0, index_col=0)
y_final_test = pd.read_csv('../data/processed/y_test', header=0, index_col=0)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(189, 13) (189, 1) (48, 13) (48, 1)


In [54]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [65]:
model = 'Random Forest Classfier'
for n in (10,20,30,40,50,100):
    for d in (2,3,4,5,6,7,8):
        rf = RandomForestClassifier(n_estimators=n, random_state=42,max_depth=d)
        rf.fit(X_train,y_train)
        rf_predicted = rf.predict(X_test)
        rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
        rf_acc_score = accuracy_score(y_test, rf_predicted)
        print("confusion matrix")
        print(rf_conf_matrix)
        print("\n")
        print(f"Accuracy of Random Forest with {n} estimators and {d} depth:{rf_acc_score*100} \n")
        print(classification_report(y_test,rf_predicted))

confusion matrix
[[23  1]
 [ 7 17]]


Accuracy of Random Forest with 10 estimators and 2 depth:83.33333333333334 

              precision    recall  f1-score   support

           0       0.77      0.96      0.85        24
           1       0.94      0.71      0.81        24

    accuracy                           0.83        48
   macro avg       0.86      0.83      0.83        48
weighted avg       0.86      0.83      0.83        48

confusion matrix
[[20  4]
 [ 6 18]]


Accuracy of Random Forest with 10 estimators and 3 depth:79.16666666666666 

              precision    recall  f1-score   support

           0       0.77      0.83      0.80        24
           1       0.82      0.75      0.78        24

    accuracy                           0.79        48
   macro avg       0.79      0.79      0.79        48
weighted avg       0.79      0.79      0.79        48

confusion matrix
[[21  3]
 [ 6 18]]


Accuracy of Random Forest with 10 estimators and 4 depth:81.25 

             

In [59]:
rf2 = RandomForestClassifier(n_estimators=20, random_state=42,max_depth=5)
rf2.fit(X_train,y_train)
scores = cross_val_score(rf, X, y.values.ravel(), cv=5)
scores

array([0.83333333, 0.875     , 0.80851064, 0.76595745, 0.89361702])